Adaptive basis-function Model (ABM) has the form

$$f(x) = w_0 + \sum_{m=1}^{M}w_m\phi_m(x)$$

where $\phi_m$ is the $m^{th}$ basis function learned from the data

# Classification & Regression Trees

The model is of the following form

$$f(x) = E[y|x] = \sum_{m=1}^M w_m \mathbb{I}(x \in R_m) = \sum_{m=1}^Mw_m\phi_m(x'v_m)$$

where $R_m$ is the $m^{th}$ region $w_m$ is the mean response of the region and $v_m$ encodes the choice of variable to split on and threshold value on path from the root to the $m^{th}$ leaf

CART is an ABM where basis functions define the regions and weights specify the respose value in each region. When classifying store the distribution over class labels in each leaf instead of the mean response.

E.G see figure 16.1

# Growing A Tree

optimal partitioning is np complete so it's common to use a greedy procedure to compute locally optimal MLE. The split function choose best feature and best value for the feature by

$$(j^*,t^*) = argmin_{j \in \{1 \cdots D\}} \; min_{t \in \tau_j}\; cost(\{x_i,y_i :x_{ij} \leq t \}) + cost(\{x_i,y_i : x_{ij} >t\})  $$

its assumed all inputs are real valued or ordinal so we can compare feature $x_{ij}$ to a numeric value $t$ the set of all possible thresholds $\tau_j$ for feature $j$ is obtained by sorting uniq values of $x_{ij}$ E.G if feature one has values $\{4.5,-12,72,-12\}$ then $\tau_1 = \{-12,4.5,72\}$

for categorical inputs consider splits of form $x_{ij} = c_k$ and $x_{ij} \neq c_x$ for each class label $c_k$. note: could do multiway splits but often this leads to too little data in each subtree and thus overfitting

Heuristics for checking if a node is worth fitting

1. is the reduction in cost too small, typically the gain of using a feature is a normalized measure of the reduction in cost

$$\Delta \triangleq cost(D) - \left(\frac{|D_L|}{|D|}\;cost(D_L) + \frac{|D_R|}{D} \; cost(D_R)\right)$$

2. has to tree exceeded maximum desired depth
3. is the distribution of the response in either $D_L$ or $D_k$ sufficiently homogenous EG all labels are the same
4. is the number of examples in either $D_L$ or $D_k$ too small


## Regression Cost

$$cost(D) = \sum_{i \in D} (y_i - \bar{y})$$

you could also fit a linreg model for each leaf using as inputs the features that were chosen on the path from the root then measure residual error

## Classification Cost

to measure quality of a split fit a multinouli model on leaf satisfying test $ X_j <t$ by estimating class conditional probabilities

$$\hat{\pi} = \frac{1}{|D|} \sum_{i \in D} \; \mathbb{I}(y_i = c)$$
where $D$ is the data in the leaf

common error measures for evaluating a parititon
1. missclassification rate define most probable class label as $\hat{y_c}= argmax_c \hat{\pi_c}$ then the error rate is

$$\frac{1}{|D|} \sum_{i \in D} \mathbb{I} (y_i \neq \hat{y}) = 1-\hat{\pi}_{\hat{y}}$$

2. entropy or deviance

$$\mathbb{H}(\hat{\pi}) = -\sum_{c=1}^{C} \hat{\pi_c}log(\hat{\pi_c})$$

minimizing entropy maximizes information gain

3. Gini Index

$$\sum_{c=1}^C \hat{\pi_c}(1-\hat{\pi_c}) = 1-\sum_c \hat{\pi_c}^2$$ where $\hat{\pi_c}$ is the pobability a random entry in a leaf belongs to class $c$


## Pruning a Tree
grow a full tree then prune it, to determine how far back to prune evaluate cross-validated error on each subtree then pick the tree whose CV error is within 1 SD of the minimum

## Random Forests

one way to reduce variance is to average many estimates for example grow $M$ different trees on different subsets of the data chose randomly with replacement then compute ensemble

$$fx) = \sum_{m=1}^M \frac{1}{M} f_m(x)$$ where $f_m$ is the $m^{th}$ tree this is known as bagging

runing same leaerning on different subsets results in highly correlated predictors that limits that possible variance reduction. random forests tries to decorrelate base learners by learning trees based on randomly chosen subset of input variables as well as randomly chosen subset of data cases.

In [3]:
#data import
from sklearn import datasets
import pandas as pd
from sklearn.model_selection import train_test_split



#load dataset convert to df
iris = datasets.load_iris()
data=pd.DataFrame({
    'sepal length':iris.data[:,0],
    'sepal width':iris.data[:,1],
    'petal length':iris.data[:,2],
    'petal width':iris.data[:,3],
    'species':iris.target
})

#set features labes train test split
X=data[['sepal length', 'sepal width', 'petal length', 'petal width']]  # Features
y=data['species']  # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

/home/brandons/anaconda3/envs/minerva/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/brandons/anaconda3/envs/minerva/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [4]:
#fit random forest classifier
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

/home/brandons/anaconda3/envs/minerva/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/brandons/anaconda3/envs/minerva/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/brandons/anaconda3/envs/minerva/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [5]:
#test accuracy
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


Accuracy: 0.9736842105263158


In [6]:
#make prediction
species_idx = rfc.predict([[3, 5, 4, 2]])[0]
iris.target_names[species_idx]

'virginica'

In [12]:
#look at feature importance
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

feature_imp = pd.Series(rfc.feature_importances_,index=iris.feature_names)
sns.barplot(x=feature_imp, y=feature_imp.index)
plt.xlabel('Feature Importance Score')
plt.ylabel('Features')
plt.title("Visualizing Important Features")
plt.legend()

AttributeError: module 'seaborn' has no attribute 'barplot'

# Generalized Additive Models

has the form

$$f(x) - \alpha + f_1(x_1) + \cdots + f_D(x_D)$$

each $f_j$ can be modeled by some scatterplot smoother and $f(x)$ can be mapped to $p(y|x)$ using a link function

it is typical to use smoothing splines for $f_j$ in this case the objective becomes

$$J(\alpha, f_1, \cdots , f_d) = \sum_{i=1}^N \left(y_i - \alpha - \sum_{j=1}^D f_j(x_{ij}) \right)^2 + \sum_{j=1}^D \lambda_j \int f_j^{''}(t_j)^2dt_j$$ where $\lambda_j$ is the strength of the regularizer for $f_j$

# Multivariate Adaptive Regression Splines (MARS)

extends GAM's by allowing for interaction effects with an ANOVA like decomposition

$$f(x)=\beta_0+\sum_{j=1}^Df_j(x_j)+\sum_{j,k}f_{jk}(x_j,x_k)+\sum_{j,k,l}f_{jkl}(x_j,x_k,x_l)+\cdots$$

mars uses a greedy search to decide which variables to add, fitting tensor product basis of regression splines to represent multidimensional regression functions EG for 2D input

$$f(x_1,x_2)=\beta_0+\sum_m\beta_{1m}(x_1-t_{1m})+\sum_m\beta_{2m}(t_{2m}-x_2)+\sum_m\beta_(12m(x_1-t_{1m})+(t_{2m}-x_2)+$$

to create function start with candidate set of basis functions of form

$$\mathcal{C} = \{(x_j-t)_+,(t-x_j)_+:t \in \{x_{1j},\cdots,x_{mj},j=1,\cdots ,D\}\}$$

these are 1d linear splines where knots are at all observed values of the variables, consider both reflecting pairs (splines sloping up and splines sloping down)

let $\mathcal{M}$ be the current set of basis functions we initialize with $\mathcal{M}=\{1\}$ and to create a new basis function pair multiply by an $h_m\in \mathcal{M}$ with one of the reflecting pairs in $\mathcal{C}$

eg say initially we get $f(x)=25-4(x_1-5)_++20(5-x_1)_+$ obtained by multiplying $h_0(x)=1$ with a reflecting pair involving $x_1$ with knot $t=5$ this pair is added to $\mathcal{M}$ at the next step we might have

$$f(x)=2-2(x_1-5)_+ +3(5-x_1)_+-(x_2-10) \times (5-x_1)+(-1.2(10-x_2)\times (5-x_1)_+ $$ obtained by multiplying $(5-x_1)+$ from $\mathcal{M}$ by new reflecting pair $(x_2-10)_+$ and $(10-x_2)_+$


continue until model gets pretty big then prune eliminating basis function that causes smallest increase in residual error until CV error stops improving.

# Boosting

Boosting attempts to solve

$$\underset{f}{min}\sum_{i=1}^{N}L(y_i,f(x_i))$$ for some loss function $L$

1: Squared Error loss

if we use squared error loss the optimal estimate is given by

$$f^*(x)=\underset{f(x)}{argmin}\;\mathbb{E}_{y|x}\left[(Y-f(x))^2\right]=\mathbb{E}[Y|x]$$

2. log/exponential loss

for binary classificatoin use logloss (the convex upper bound of 0-1 loss) the optimal estimate is

$$f^*(x)=\frac{1}{2}log\frac{p(\tilde{y}=1|x)}{p(\tilde{y}=-1|x)}$$

an alternate convex upper bound is log loss ($exp(-\tilde{y}f)$ whose optimal estimate is $e^{2f(x)}$

in both cases we try to approzimate (half) the log odds ratio, since finding optimal $f$ is hard its tackled sequentially, first we initialize

$$f_0(x)=\underset{\gamma}{argmin}\sum_{i=1}^NL(y_i, f_{m-1}(x_i)+\beta\phi(x_i;\gamma))$$

we then set

$$f_m(x)=f_{m-1}(x)+\beta_m\phi(x;\gamma_m)$$

we dont go back and adjust earlier parameters, thats why its forward stagewise addive modelling

in practice better test set performance can be obtained by performing partial updates of the form

$$f_m(x)=f_{m-1}(x)+\mathcal{v}\beta_m\phi(x;\gamma_m)$$ where shrinkage parameter $0<\mathcal{v}\leq 1$ typically set it to small value $\mathcal{v}=0.1$

# L2 Boosting

suppose we used squared error loss then at step $m$ the loss has the form

$$L(y_i;f_{m-1}(x)+\beta \phi(x_i,\gamma))=(r_{im}-\phi(x_i;\gamma))^2$$

where $r_{im} \triangleq y_i-f_{m-1}(x_i)$ is the current residual we can use a weak learner to learn $r_m$

# Gradient Boosting

we try to minimize $\hat{f}=\underset{f}{argmin}\;L(f)$ where $f=(f(x_1),\cdots , f(x_N))$ are the parameters we solve with stagewise gradient descent at stage $m$ let $g_m$ be the gradient of $L(f)$ evaluated at $f=f_{m-1}$

$$g_{im}=\left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}\right]_{f=f_{m-1}}$$

![Loss Function Table]("loss_function_table.jpg")

we then make an update $f_m=f_{m-1}-\rho_mg_m$ where $\rho_m$ is the step length chosen by $\rho_m=\underset{\rho}{argmin}L(f_{m-1}-\rho g_m)$ in this form it only optimizes $f$ at a fixed set of $N$ points we can modify the algorithm by fitting a weak learner to approximate the negative gradient signal

$$\gamma_m=\underset{\gamma}{argmin}\sum_{i=1}^N (-g_{im}-\phi(x_i;\gamma))^2$$

![Gradient Boosting]("gradient_boosting_algo.jpg")

# Gradient Boosting

we try to minimize $\hat{f}=\underset{f}{argmin}\;L(f)$ where $f=(f(x_1),\cdots , f(x_N))$ are the parameters we solve with stagewise gradient descent at stage $m$ let $g_m$ be the gradient of $L(f)$ evaluated at $f=f_{m-1}$

$$g_{im}=\left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}\right]_{f=f_{m-1}}$$

![Loss Function Table]("loss_function_table.jpg")

we then make an update $f_m=f_{m-1}-\rho_mg_m$ where $\rho_m$ is the step length chosen by $\rho_m=\underset{\rho}{argmin}L(f_{m-1}-\rho g_m)$ in this form it only optimizes $f$ at a fixed set of $N$ points we can modify the algorithm by fitting a weak learner to approximate the negative gradient signal

$$\gamma_m=\underset{\gamma}{argmin}\sum_{i=1}^N (-g_{im}-\phi(x_i;\gamma))^2$$

![Gradient Boosting]("gradient_boosting_algo.jpg")

# Gradient Boosting

we try to minimize $\hat{f}=\underset{f}{argmin}\;L(f)$ where $f=(f(x_1),\cdots , f(x_N))$ are the parameters we solve with stagewise gradient descent at stage $m$ let $g_m$ be the gradient of $L(f)$ evaluated at $f=f_{m-1}$

$$g_{im}=\left[\frac{\partial L(y_i,f(x_i))}{\partial f(x_i)}\right]_{f=f_{m-1}}$$

![Loss Function Table]("loss_function_table.jpg")

we then make an update $f_m=f_{m-1}-\rho_mg_m$ where $\rho_m$ is the step length chosen by $\rho_m=\underset{\rho}{argmin}L(f_{m-1}-\rho g_m)$ in this form it only optimizes $f$ at a fixed set of $N$ points we can modify the algorithm by fitting a weak learner to approximate the negative gradient signal

$$\gamma_m=\underset{\gamma}{argmin}\sum_{i=1}^N (-g_{im}-\phi(x_i;\gamma))^2$$

![Gradient Boosting]("gradient_boosting_algo.jpg")